<a href="https://colab.research.google.com/github/eric15342335/realfill/blob/main/train_realfill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/eric15342335/realfill
%cd realfill

Cloning into 'realfill'...
remote: Enumerating objects: 276, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 276 (delta 41), reused 30 (delta 23), pack-reused 216 (from 1)
Receiving objects: 100% (276/276), 1.25 MiB | 29.18 MiB/s, done.
Resolving deltas: 100% (112/112), done.
/content/realfill


In [9]:
!curl -L https://github.com/eric15342335/realfill/releases/download/dataset/realfill_data_release_full.zip -o realfill_data_release_full.zip
!unzip -q realfill_data_release_full.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  452M  100  452M    0     0  28.7M      0  0:00:15  0:00:15 --:--:-- 29.1M


In [10]:
%pwd
%ls -lh

total 454M
drwxr-xr-x 4 root root 4.0K Apr 19 15:33 data/
-rw-r--r-- 1 root root 2.5K Apr 19 15:33 infer.py
-rw-r--r-- 1 root root 1.1K Apr 19 15:33 LICENSE
drwxr-xr-x 3 root root 4.0K Apr 19 15:37 __MACOSX/
-rw-r--r-- 1 root root  383 Apr 19 15:33 README.md
-rw-r--r-- 1 root root 5.9K Apr 19 15:33 README-Realfill.md
drwxr-xr-x 4 root root 4.0K May 30  2024 realfill_data_release_full/
-rw-r--r-- 1 root root 453M Apr 19 15:37 realfill_data_release_full.zip
-rw-r--r-- 1 root root   97 Apr 19 15:33 requirements.txt
-rw-r--r-- 1 root root 4.8K Apr 19 15:33 train_realfill.ipynb
-rw-r--r-- 1 root root  37K Apr 19 15:33 train_realfill.py


In [12]:
%pip install -U -r requirements.txt --progress-bar off

In [13]:
from accelerate.utils import write_basic_config
write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [19]:
import os
os.environ["DATASET"] = "realfill_data_release_full"
os.environ["MODEL_NAME"] = "stabilityai/stable-diffusion-2-inpainting"
os.environ["BENCHMARK"] = "RealBench" # Replace this with "Qualitative"
os.environ["DATASET_NUMBER"] = "23"
os.environ["TRAIN_DIR"] = f'{os.environ["DATASET"]}/{os.environ["BENCHMARK"]}/{os.environ["DATASET_NUMBER"]}'
os.environ["OUTPUT_DIR"] = f'{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}-model'
os.environ["VAL_IMG"] = f'{os.environ["TRAIN_DIR"]}/target/target.png'
os.environ["VAL_MASK"] = f'{os.environ["TRAIN_DIR"]}/target/mask.png'
os.environ["OUTPUT_IMG_DIR"] = f'{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}-results'

In [ ]:
!accelerate launch train_realfill.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --train_data_dir=$TRAIN_DIR \
  --output_dir=$OUTPUT_DIR \
  --resolution=512 \
  --train_batch_size=16 \
  --gradient_accumulation_steps=1 \
  --unet_learning_rate=2e-4 \
  --text_encoder_learning_rate=4e-5 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=100 \
  --max_train_steps=2000 \
  --lora_rank=8 \
  --lora_dropout=0.1 \
  --lora_alpha=16 \
  --resume_from_checkpoint="latest" \
  --report_to tensorboard \
  --validation_steps 100 \
  --checkpointing_steps 100 \
  --mixed_precision="bf16"

In [ ]:
!accelerate launch infer.py \
    --model_path=$OUTPUT_DIR \
    --validation_image=$VAL_IMG \
    --validation_mask=$VAL_MASK \
    --output_dir=$OUTPUT_IMG_DIR

In [ ]:
# Zip final inference results
!zip -r $OUTPUT_IMG_DIR.zip $OUTPUT_IMG_DIR
# Zip tensorboard logs
!zip -r $OUTPUT_DIR-tensorboard.zip $OUTPUT_DIR/logs